## 1. Import dependencies

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sha2

In [0]:
df_silver = spark.read.json("/mnt/bronze/breweries")

In [0]:
df_silver.display()

address_1,address_2,brewery_type,city,country,id,latitude,longitude,name,phone,postal_code,state,state_province,street,website_url
1716 Topeka St,null,micro,Norman,United States,5128df48-79fc-4f0f-8b52-d06be54d0cec,35.25738891,-97.46818222,(405) Brewing Co,4058160490,73069-8224,Oklahoma,Oklahoma,1716 Topeka St,http://www.405brewing.com
407 Radam Ln Ste F200,null,micro,Austin,United States,9c5a66c8-cc13-416f-a5d9-0a769c87d318,null,null,(512) Brewing Co,5129211545,78745-1197,Texas,Texas,407 Radam Ln Ste F200,http://www.512brewing.com
8100 Washington Ave,null,micro,Mount Pleasant,United States,34e8c68b-6146-453f-a4b9-1f6cd99a5ada,42.72010826899558,-87.88336350209435,1 of Us Brewing Company,2624847553,53406-3920,Wisconsin,Wisconsin,8100 Washington Ave,https://www.1ofusbrewing.com
1501 E St,null,large,San Diego,United States,ef970757-fe42-416f-931d-722451f1f59c,32.714813,-117.129593,10 Barrel Brewing Co,6195782311,92101-6618,California,California,1501 E St,http://10barrel.com
62970 18th St,null,large,Bend,United States,6d14b220-8926-4521-8d19-b98a2d6ec3db,44.08683531,-121.281706,10 Barrel Brewing Co,5415851007,97701-9847,Oregon,Oregon,62970 18th St,http://www.10barrel.com
1135 NW Galveston Ave Ste B,null,large,Bend,United States,e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,44.0575649,-121.3288021,10 Barrel Brewing Co,5415851007,97703-2465,Oregon,Oregon,1135 NW Galveston Ave Ste B,null
1411 NW Flanders St,null,large,Portland,United States,e432899b-7f58-455f-9c7b-9a6e2130a1e0,45.5259786,-122.6855056,10 Barrel Brewing Co,5032241700,97209-2620,Oregon,Oregon,1411 NW Flanders St,http://www.10barrel.com
62950 NE 18th St,null,large,Bend,United States,9f1852da-c312-42da-9a31-097bac81c4c0,44.0912109,-121.2809536,10 Barrel Brewing Co - Bend Pub,5415851007,97701,Oregon,Oregon,62950 NE 18th St,null
826 W Bannock St,null,large,Boise,United States,ea4f30c0-bce6-416b-8904-fab4055a7362,43.618516,-116.202929,10 Barrel Brewing Co - Boise,2083445870,83702-5857,Idaho,Idaho,826 W Bannock St,http://www.10barrel.com
2620 Walnut St,null,large,Denver,United States,1988eb86-f0a2-4674-ba04-02454efa0d31,39.7592508,-104.9853655,10 Barrel Brewing Co - Denver,7205738992,80205-2231,Colorado,Colorado,2620 Walnut St,null


#2. Transform data

Applying hash to sensitive data

In [0]:
df_silver_hashed = df_silver \
    .withColumn("phone", sha2(col("phone"), 256)) \
    .withColumn("street", sha2(col("street"), 256)) \
    .withColumn("website_url", sha2(col("website_url"), 256)) \
    .withColumn("postal_code", sha2(col("postal_code"), 256)) \
    .select(
        col('id').alias("brewery_id"),
        col('name').alias("brewery_name"),
        col('brewery_type'),
        col('address_1'),
        col('address_2'),
        col('city'),
        col('state_province'),
        col('postal_code'),
        col('country'),
        col('longitude'),
        col('latitude'),
        col('phone'),
        col('website_url'),
        col('state'),
        col('street')
    )

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;

#3. Loading as a delta table

Delta tables are more performatic than parquets

In [0]:
df_silver_hashed.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("country") \
    .saveAsTable("silver.tbl_breweries")

In [0]:
%sql

select * from silver.tbl_breweries

brewery_id,brewery_name,brewery_type,address_1,address_2,city,state_province,postal_code,country,longitude,latitude,phone,website_url,state,street
5128df48-79fc-4f0f-8b52-d06be54d0cec,(405) Brewing Co,micro,1716 Topeka St,null,Norman,Oklahoma,96716d2c948d6a35109f3d92931f87e20f066584e31c985097c5901312b92cc3,United States,-97.46818222,35.25738891,9c7560fa4ce8fa49571ae192fa012071cddb83acd619b88ffb0fa34e187afedb,200a50b942b628775a1996db084b0333a913cda8f65f59ae71015276cb90a64a,Oklahoma,f11708d4ce5286f83d22ddceac7703279e8ba669706b690f08c86b09eb4810ce
9c5a66c8-cc13-416f-a5d9-0a769c87d318,(512) Brewing Co,micro,407 Radam Ln Ste F200,null,Austin,Texas,1fd73f19effcbea73fa56d6271c1fe4e276d8433e713e3e2fc118467a6fe24de,United States,null,null,1bd4e6cd2464fb71f41cccd4f3fa2a0ac3ddbe953d3918c5e8c978f644e45738,b23ae6858de9b6b743ef64a0b84e419fea3d519f2bd78600fad309f578115ec8,Texas,dca4f8da0181dffc45df24ea9e8ebc63fbf71d0357e13dbe6aea20473b4fac28
34e8c68b-6146-453f-a4b9-1f6cd99a5ada,1 of Us Brewing Company,micro,8100 Washington Ave,null,Mount Pleasant,Wisconsin,cbf549f11ea8c74e7f38967bb3d6ba880b9c2458391e4bb6957bce1118902344,United States,-87.88336350209435,42.72010826899558,9f180b18d7efaa409e20a0b86a67fef0de02ba9c3f18efc7057871c8e451cf25,3c207262654c98cb3a61a2cd682d7aee221ad9653c75404adb11b612058d8958,Wisconsin,99cdd3ef12f191e9b09ba628cb765367739927278f3fe532a18fecba5579f05e
ef970757-fe42-416f-931d-722451f1f59c,10 Barrel Brewing Co,large,1501 E St,null,San Diego,California,508e032204eb9d4172a1aa0443e22181eb5a7eb85f359367c1827e2325b4a972,United States,-117.129593,32.714813,5799031d5ac5d84dee4cc4de1dd04726af1514ad9f477ca5b09703a9b31a9f7d,cbdb65f25839ce7cb45de1524d70490982f82632b2a0ba4313967557763d9be1,California,b2332ce5dd82beb297ce8613c36938e578dd0fc24151b67d9737fa15eeb13626
6d14b220-8926-4521-8d19-b98a2d6ec3db,10 Barrel Brewing Co,large,62970 18th St,null,Bend,Oregon,32d18bf11794cb66b4065af088c5e93c149bac18595e6e743e5e660d1dc310d7,United States,-121.281706,44.08683531,06a9837eb6c5c12a231a47865f1ee79a62c5b3778fe8925f32684dc45d4d79f9,a403be3fda7fc511298a70fe6c32e1788230f054e6b5bf88bce46dd180689cf5,Oregon,5e1879ede87f8c2dd9b7bffd6fa35f5200c1262bd444462821fac9ea8b8a46e5
e2e78bd8-80ff-4a61-a65c-3bfbd9d76ce2,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,null,Bend,Oregon,b5e738fbb75af64cb92b7d14e9be1f00756fb30c03f1547b7e4b8087789b9b9e,United States,-121.3288021,44.0575649,06a9837eb6c5c12a231a47865f1ee79a62c5b3778fe8925f32684dc45d4d79f9,null,Oregon,cb8e323e1ebb910c226dbb0a9778b3290e2beafacc48e9592bad03102cf9dc15
e432899b-7f58-455f-9c7b-9a6e2130a1e0,10 Barrel Brewing Co,large,1411 NW Flanders St,null,Portland,Oregon,d46033a9a59d8e0f27797cf296d67d1a3f3e37b93bfed84deaf1a9de914b354a,United States,-122.6855056,45.5259786,c9b7ca8a5897720f4232729023818a5c0d2ec12ae9fee49d2ecba0e8f26938d5,a403be3fda7fc511298a70fe6c32e1788230f054e6b5bf88bce46dd180689cf5,Oregon,677cce4d2a59fc75dc404b47175e73ec8d2eae5653f008a4968a1e5f18a829be
9f1852da-c312-42da-9a31-097bac81c4c0,10 Barrel Brewing Co - Bend Pub,large,62950 NE 18th St,null,Bend,Oregon,4d42bdf0c995a7c6482e8b7948d24815d54689ca2dde1d4c53bb82a80595fdca,United States,-121.2809536,44.0912109,06a9837eb6c5c12a231a47865f1ee79a62c5b3778fe8925f32684dc45d4d79f9,null,Oregon,dae95dff1fd2a4a81244140eb8fd978e72c20c3fdfd6da862a6c90a4f7930062
ea4f30c0-bce6-416b-8904-fab4055a7362,10 Barrel Brewing Co - Boise,large,826 W Bannock St,null,Boise,Idaho,a7458ee425cd129ee5225c47fe5a22a1d5df1af4e035f3e05a97c0336ecfe7c4,United States,-116.202929,43.618516,f2cf20597c778a990fcccd5f593b4e56a4d1c6c922a34455e18dc51377884aaa,a403be3fda7fc511298a70fe6c32e1788230f054e6b5bf88bce46dd180689cf5,Idaho,8bc077f08f43058ecbe3d055f704cf072ad6afdf93bf869e33c7b502b1877331
1988eb86-f0a2-4674-ba04-02454efa0d31,10 Barrel Brewing Co - Denver,large,2620 Walnut St,null,Denver,Colorado,c3dc001d59dab8c6baacb7078a08c08401f0ec10e2445c4061c3ccb0be42aea4,United States,-104.9853655,39.7592508,b8e2519e3b7332dec66aa36f8c58be0c3fcd52eb2458ae576e84bcaf47db14cf,null,Colora